# Data Cleaninsing with Spark

*Note: This is a prototype implementation of the data cleaning process from clean.py in PySpark. This is still under construction due to several issues and bugs.* 

In [ ]:
%pip install pyspark

: 

In [ ]:
%pip install sparknlp

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext, Row
import pyspark.sql.functions as f
from pyspark import SparkContext
from sparknlp.pretrained import PretrainedPipeline
from sparknlp import Finisher
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from sparknlp.base import *
from sparknlp.annotator import *
import pandas as pd
from langdetect import detect
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
import re
import os

In [2]:
spark = SparkSession.builder\
   .appName("PythonNLPAPPLIER")\
   .getOrCreate()

# sqlContext = SQLContext(spark)
# spark = SparkSession.builder \
#     .appName("nlp converter")\
#     .master("local[*]")\
#     .config("spark.driver.memory","16G")\
#     .config("spark.driver.maxResultSize", "300") \
#     .config("spark.kryoserializer.buffer.max", "2000M")\
#     .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.2.6")\
#     .getOrCreate()
# spark

In [3]:
# path = r"files\raw_texts.feather"
# pandas_df =  pd.read_feather(source+path).drop_duplicates(subset = 'URL', keep = 'first').reset_index(drop=True)
# pandas_df = pandas_df[pandas_df['URL_TEXT']!=""]
# spark_df = sqlContext.createDataFrame(pandas_df)

source = os.path.dirname(os.path.realpath('__file__')).split("src")[0]
path = r"files\raw_texts.parquet"
path_en = r"files\raw_texts_en.parquet"

spark_df = spark.read.parquet(source+path).where(f.col("URL_TEXT")!= "")
print(spark_df.count())
# spark_df_en = spark.read.parquet(source+path_en).where(f.col("URL_TEXT")!= "")
# print(spark_df_en.count())

38554


In [5]:
def clean_text(spark_df):
    xml = ["(?:<from.*?>)(.*?)(?:<\\/from>)"]
    html = ["<(?:\"[^\"]*\"['\"]*|'[^']*'['\"]*|[^'\">])+>"]        
    random_pattern = ['^@.*\{.*\}', '^\..*\{.*\}','\s\s+','\xa0','dbx707', '\xe2','\x80',"\x8b", r"\{\{\.*\}\}", "\x9d", "\u200b"]# only digits: r'\b[0-9]+\b\s*'
    url = ["^https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[äöüßa-zA-Z0-9()]{1,6}\\b(?:[-a-zäöüßA-Z0-9()@:%_\\+.~#?&\\/=]*)$", "www\w*de","www\w*com"]
    email = ["^\S+@\S+\.\S+$"]
    zip = ["^[0-9]{5}(?:-[0-9]{4})?\s?\w*$"]
    phone = ["^\\+?[1-9][0-9]{7,14}$"]
    dates = ["^[0-9]{1,2}\\/[0-9]{1,2}\\/[0-9]{4}$","^[0-9]{1,2}\\-[0-9]{1,2}\\-[0-9]{4}$", "^[0-9]{4}\\-[0-9]{1,2}\\-[0-9]{1,2}$"]
    website_stopwords = ["explore","allgemeine geschäftsbedingungen","allgemein\*",'richtlinie\w*',"\w*recht\w* hinweis\w*","\w*recht\w*","\w*datenschutz\w*", "privacy","policy\w*","cooky\w*","cookie\w*","content\w*"," to ",\
            "anmeld\w*",  "abmeld\w*", "login","log in","logout", "log out", "kunden login", "online","zurück","back","start","select\w*", "ausw\w*","close",\
                "extras","news","report\w*","impressum","newsletter\w*", "owner","internet", "website\w*", "email\w*", "e-mail\w*", "mail\w*", "isbn", "issn",\
                    "produkte", "partner","übersicht", "veranstaltungen", "suche\w*","kauf\w*", "angebot\w*", "konfigur\w*", "configur\w*","nutzer\w*","icon\w*",\
                        "zubehör", "garantie", "mehr", "modell\w*", "kontakt\w*","contact\w*","anfrage\w*","skip",'useful links','link\w*',"pin\w*","passw\w*", "password\w*",\
                            "buchen","book" "anfahrt", "finanzdienstleistung\w*" "connected", "required", "sitemap\w*", "\w*\s?abo\w*", 'social media', "socialmedia",\
                                "englisch", "english","deutsch","german","google", "wikipedia", "navigation","\w*shop\w*", "\w*magazin\w*", "lifestyle",\
                                    "facebook\w*", "youtube\w*","instagram\w*","xing\w*","linkedin\w*", "blog\w*","spiegel\w*","twitter\w*","sms","video"\
                                        "archiv\w*", "artikel\w*", "article\w*","side\w*", "seite\w*","site","app\w*","\s?abgerufen\s?\w*\s*\d*",\
                                            "januar", "februar", "märz", "april", "mai", "juni", "juli", "august", "september", "oktober", "november", "dezember",\
                                                "january", "feburary", "march","may", "june", "july","septembre", "octobre", "novembre", "decembre",\
                                                "dbx707", "db11","\w*\s?straße\s?\d*","\w*\s?strasse\w*", "tel\w*", "\w*\s?download\w*",\
                                                    "covid\w*\s?\d*", "corona\w*\s?\d*"]
                                    
    domain_stopwords = ["(g/km)","use case\w*", "unternehme\w*", "gmbh", "cokg", "co kg", "consult\w*", "handel\w*", "händler\w*", "leistung\w*"]
    numbers_only = ["^\\d+$","^\s?[0-9]+(\s+[0-9]+)*\s?$", "\(.*\)","\[.*\]", "^\d+.\d+"," \\d+ "]
    special_characters = ['[^äöüßA-Za-z0-9 ]+']#['[\(,.:\);^]']
    short_words = ['^\.{0,3}$']

    all_pattern_to_remove = email+xml+html+random_pattern+url+zip+phone+dates+website_stopwords+domain_stopwords+numbers_only+special_characters+short_words
    spark_df = spark_df.withColumnRenamed("URL_TEXT", 'text')
    spark_df = spark_df.withColumn('text', f.lower(f.col('text')))
    spark_df = spark_df.withColumn('text', f.regexp_replace(f.col('text'), '\r+|\n+|\t+/', ''))
    spark_df = spark_df.withColumn('text', f.regexp_replace(f.col('text'),'[\|]+', ' '))
    spark_df = spark_df.withColumn('text', f.regexp_replace(f.col('text'),'\s\s+', ' '))
    for pattern in all_pattern_to_remove:
        spark_df = spark_df.withColumn("text", f.regexp_replace(f.col('text'), pattern, ""))
    spark_df = spark_df.withColumn('text', f.regexp_replace(f.col('text'),'\s\s+', ' ')) 

    return spark_df

def tokenize_text(spark_df):
    regexTokenizer = RegexTokenizer(pattern='\w+', inputCol="text", outputCol="words")
    spark_df = regexTokenizer.transform(spark_df)
    return spark_df
    

def detect_language(text:str) -> str:
        """Helper Function for detecting language of row using langdetect package.
        Args:
            text (str): one sample (row) containing text of one crawled website.

        Returns:
            str: detected language as unicode.
        """
        return_lan = None
        try:
            return_lan = str(detect(text)).lower()
        except: 
            return_lan = None
        return return_lan    

langUDF = udf(lambda row: detect_language(row),StringType())


In [6]:
spark_df = clean_text(spark_df)
# spark_df = spark_df.withColumn("LANG", langUDF(f.col('text')))
# spark_df_en = clean_text(spark_df_en)
# spark_df_en = spark_df_en.withColumn("LANG", langUDF(f.col('text')))
# spark_df_en.show(truncate=False)
spark_df.show(truncate=False)

+-------+------------+---------------------+------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
spark_df = spark_df.withColumn("LANG", langUDF(f.col('text')))
spark_df.show(truncate=False)


Py4JJavaError: An error occurred while calling o575.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 10) (DESKTOP-FD0BJOT.fritz.box executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:855)
	at org.apache.spark.rdd.RDD.$anon

In [ ]:
spark_df.count()

In [ ]:
spark_sample = spark_df.sample(False, 0.005,42)

In [ ]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

language_detector = LanguageDetectorDL.pretrained("ld_wiki_tatoeba_cnn_21")\
    .setInputCols(["document"])\
    .setOutputCol("lang")\
    .setThreshold(0.8)\
    .setCoalesceSentences(True)

sentenceDetector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("Sentence")

regexTokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

finisher = Finisher() \
    .setInputCols(["token"]) \
    .setIncludeMetadata(True)

pipeline = Pipeline() \
.setStages([
    documentAssembler,
    language_detector,
    sentenceDetector,
    regexTokenizer,
    finisher
    ])

result = pipeline.fit(spark_df).transform(spark_df)

In [ ]:
spark_df.head()